In [6]:
#The import cell: You'll only need to run this cell once, but you'll need to run it first
# To run, hold shift and press Enter.
import hoomd
import hoomd.md
from hoomd import deprecated
import mbuild, mdtraj, nglview, numpy
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
# The 'settings' cell
# You can change these values to change how the simulations run
Number_of_particles = 900 #A quirk of this notebook is that you should choose a square number here
Temperature = 2. #These are "dimensionless temperature units" that map to Kelvin based upon the Lennard-Jones parameters

In [8]:
# The simulation engine initializer cell
# This if you change something in the settings above, you'll need to re-run that cell and this one before 
# running another simulation
hoomd.context.initialize("")
system=hoomd.init.create_lattice(unitcell=hoomd.lattice.unitcell(N=1,a1=[1.1,0,0],a2=[0,1.1,0],a3=[0,0,1],dimensions=2,type_name=["A"],position=[[0,0,0]]),n=int(Number_of_particles**.5))
deprecated.dump.xml(filename="init.hoomdxml",group=hoomd.group.all(),vis=True)
nl = hoomd.md.nlist.tree()
lj = hoomd.md.pair.lj(r_cut=2.5,nlist=nl)
lj.pair_coeff.set("A","A",epsilon=1,sigma=1)
hoomd.md.integrate.mode_standard(dt=0.007)
all=hoomd.group.all()
hoomd.md.integrate.langevin(group=all,kT=Temperature,seed=14)

notice(2): Group "all" created containing 900 particles
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for molecular simulation
  of colloidal systems using graphics processing units", Computer Physics
  Communications 203 (2016) 45--52
-----
notice(2): integrate.langevin/bd is using specified gamma values


In [9]:
#The simulation cell
# Run this cell to output a 'log' file, plus a couple of files that store info for all the microstates sampled
hoomd.dump.dcd(filename="traj.dcd",period=50,group=hoomd.group.all(),overwrite=True)
hoomd.analyze.log(filename="log",quantities=['potential_energy','temperature','pressure','kinetic_energy'],period=50,overwrite=True)
hoomd.run(5e4)
deprecated.dump.xml(filename="final.hoomdxml",group=hoomd.group.all(),vis=True)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 900
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:11 | Step 17659 / 50000 | TPS 1765.88 | ETA 00:00:18
Time 00:00:21 | Step 34842 / 50000 | TPS 1718.13 | ETA 00:00:08
Time 00:00:29 | Step 50000 / 50000 | TPS 1763.97 | ETA 00:00:00
Average TPS: 1748.53
---------
-- Neighborlist stats:
8130 normal updates / 500 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 23 / n_neigh_avg: 10.1689
shortest rebuild period: 4
** run complete **


In [10]:
# The visualization cell
# Run this to see the atoms move!
t = mdtraj.load("traj.dcd",top="final.hoomdxml")
view = nglview.show_mdtraj(t)
view.clear_representations()
view.add_representation("point",selection=[i for i in range(0,Number_of_particles//2)],color='blue',pointSize=2.5,useTexture=True)
view.add_representation("point",selection=[i for i in range(Number_of_particles//2,Number_of_particles)],color='orange',pointSize=2.5,useTexture=True)
view
#After running this cell, you can press "play" below and use the slider to view different microstates.
#Sometimes you'll get a bunch of pink warnings if the slider starts over - you can ignore these safely.

NGLWidget(count=1000)